In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load MNIST dataset
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 55.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.63MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.1MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.05MB/s]


In [ ]:
# Define CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 22 * 22, 10)
        )

    def forward(self, x):
        return self.network(x)

model = CNNModel().to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(10):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/10], Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save model
torch.save(model.state_dict(), "model_state.pth")

Epoch [1/10], Loss: 227.7634, Accuracy: 96.36%
Epoch [2/10], Loss: 80.5061, Accuracy: 98.69%
Epoch [3/10], Loss: 51.5841, Accuracy: 99.14%
Epoch [4/10], Loss: 35.2337, Accuracy: 99.41%
Epoch [5/10], Loss: 24.5463, Accuracy: 99.54%
Epoch [6/10], Loss: 17.3779, Accuracy: 99.71%
Epoch [7/10], Loss: 13.0625, Accuracy: 99.79%
Epoch [8/10], Loss: 11.6679, Accuracy: 99.78%
Epoch [9/10], Loss: 9.3733, Accuracy: 99.84%
Epoch [10/10], Loss: 8.8483, Accuracy: 99.84%


In [ ]:
# Reload the model
model = CNNModel().to(device)
model.load_state_dict(torch.load("model_state.pth"))
model.eval()

# Prediction function
def predict_image(image_path):
    img = Image.open(image_path).convert('L').resize((28, 28))
    img_tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        prediction = output.argmax(dim=1).item()

    return prediction

In [ ]:
# Predict custom images
print(predict_image('img_3.jpg'))

9


In [ ]:
# Predict custom images
print(predict_image('img_17.jpg'))

3


In [ ]:
# Predict custom images
print(predict_image('img_24.jpg'))

5
